In [1]:
import pandas as pd

# Pre-process VMIR output

In [94]:
def parse_vmir_sequence(path):
    with open(path, 'r') as f:
        txt = f.readlines()
        return txt[10].strip()
    
def get_seq_from_summary(summary, full_sequence):
    seq_start = summary.Start - 1
    sequence = full_seq[seq_start:seq_start+summary.Size]
    if summary.Orientation == 'Reverse':
        complement_trans = str.maketrans('ATGC', 'TACG')
        sequence = sequence.translate(complement_trans)
        sequence = ''.join(reversed(sequence))
    return sequence

In [6]:
export = pd.read_csv('sequence_no_header_no_whitespace_Export.txt', skiprows=32, sep='\s')
export

<ipython-input-6-3d2f9794c2cd>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  export = pd.read_csv('sequence_no_header_no_whitespace_Export.txt', skiprows=32, sep='\s')


,Rank,Name,Orientation,Start,Apex,Size,Score,Sub,HPs,Rep,HPs.1,Wind.Cnt.Abs.,Wind.Cnt.Rel.
0,121,MD1,Direct,407,445,72,137.3,0,0,3,3,NaN,NaN
1,85,MD2,Direct,443,490,97,150.6,0,0,1,1,NaN,NaN
2,194,MD3,Direct,596,640,87,120.6,0,0,1,1,NaN,NaN
3,114,MD4,Direct,651,692,75,140.3,0,0,3,3,NaN,NaN
4,95,MD5,Direct,1083,1124,86,147.3,0,0,1,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,203,MR78,Reverse,27942,27986,89,119.5,0,0,2,2,NaN,NaN
218,120,MR79,Reverse,28216,28252,81,137.4,0,0,1,1,NaN,NaN
219,73,MR80,Reverse,28421,28460,80,155.1,0,0,1,1,NaN,NaN
220,101,MR81,Reverse,28925,28966,83,145.5,0,0,1,1,NaN,NaN


In [43]:
full_seq = ''
with open('sequence_no_header_no_whitespace.fasta', 'r') as f:
    full_seq = f.read()
    
len(full_seq)

29903

In [103]:
sequences = []

for index, row in export.iterrows():
    seq = get_seq_from_summary(row, full_seq)
    sequences.append(seq)
    
export['Sequence'] = sequences
export

,Rank,Name,Orientation,Start,Apex,Size,Score,Sub,HPs,Rep,HPs.1,Wind.Cnt.Abs.,Wind.Cnt.Rel.,Sequence
0,121,MD1,Direct,407,445,72,137.3,0,0,3,3,NaN,NaN,GATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTC...
1,85,MD2,Direct,443,490,97,150.6,0,0,1,1,NaN,NaN,GTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGG...
2,194,MD3,Direct,596,640,87,120.6,0,0,1,1,NaN,NaN,GTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGA...
3,114,MD4,Direct,651,692,75,140.3,0,0,3,3,NaN,NaN,AAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGA...
4,95,MD5,Direct,1083,1124,86,147.3,0,0,1,1,NaN,NaN,TTGTATTTCCCTTAAATTCCATAATCAAGACTATTCAACCAAGGGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,203,MR78,Reverse,27942,27986,89,119.5,0,0,2,2,NaN,NaN,TACCATTTAGAATAGAAGTGAATAGGACACGGGTCATCAACTACAT...
218,120,MR79,Reverse,28216,28252,81,137.4,0,0,1,1,NaN,NaN,TTTTGGGGTCCATTATCAGACATTTTAGTTTGTTCGTTTAGATGAA...
219,73,MR80,Reverse,28421,28460,80,155.1,0,0,1,1,NaN,NaN,GTGTTAATTGGAACGCCTTGTCCTCGAGGGAATTTAAGGTCTTCCT...
220,101,MR81,Reverse,28925,28966,83,145.5,0,0,1,1,NaN,NaN,GTTTGGCCTTGTTGTTGTTGGCCTTTACCAGACATTTTGCTCTCAA...


In [107]:
export[export['Name'] == 'MD31']

,Rank,Name,Orientation,Start,Apex,Size,Score,Sub,HPs,Rep,HPs.1,Wind.Cnt.Abs.,Wind.Cnt.Rel.,Sequence
30,189,MD31,Direct,6285,6326,84,122.3,0,0,1,1,NaN,NaN,CCTGGTGTATACGTTGTCTTTGGAGCACAAAACCAGTTGAAACATC...


In [109]:
export.to_csv('pre-mirnas-with-seqs.tsv', sep='\t', index=False)

# Preprocess miRNAFold output

In [122]:
with open('miRNAfold_output.txt', 'r') as f:
    mirnas = f.read().split('\n\n')
    mirnas_seqs = [mirna.split('\n')[1].strip() for mirna in mirnas if len(mirna) > 0]
    
len(mirnas_seqs)

519

In [124]:
len(set(mirnas_seqs))

519

In [130]:
len(set(list(export['Sequence'])))

222

In [131]:
len(set(list(export['Sequence'])).intersection(set(mirnas_seqs)))

0

In [ ]:
# check complementary
complement_trans = str.maketrans('ATGC', 'TACG')
sequence = sequence.translate(complement_trans)

# check reversed

# check reversed complementary